In [ ]:
!pip install datasets evaluate transformers -qq
!pip install wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.1 MB/s eta 0:0

## 1. Business / Research Understanding Phase

## 2. Data Preparation/Understanding Phase

In [ ]:
from datasets import get_dataset_config_names, load_dataset_builder

In [ ]:
dataset_path = "amazon_reviews_multi"

configs = get_dataset_config_names(dataset_path)
print(configs)

['all_languages', 'de', 'en', 'es', 'fr', 'ja', 'zh']


In [ ]:
ds_config = 'en'
ds_builder = load_dataset_builder(dataset_path, ds_config)

print(ds_builder.info.description)
print('dataset splits: ', ds_builder.info.splits)
print('dataset features:', ds_builder.info.features)

We provide an Amazon product reviews dataset for multilingual text classification. The dataset contains reviews in English, Japanese, German, French, Chinese and Spanish, collected between November 1, 2015 and November 1, 2019. Each record in the dataset contains the review text, the review title, the star rating, an anonymized reviewer ID, an anonymized product ID and the coarse-grained product category (e.g. ‘books’, ‘appliances’, etc.) The corpus is balanced across stars, so each star rating constitutes 20% of the reviews in each language.

For each language, there are 200,000, 5,000 and 5,000 reviews in the training, development and test sets respectively. The maximum number of reviews per reviewer is 20 and the maximum number of reviews per product is 20. All reviews are truncated after 2,000 characters, and all reviews are at least 20 characters long.

Note that the language of a review does not necessarily match the language of its marketplace (e.g. reviews from amazon.de are pr

In [ ]:
from datasets import load_dataset

dataset = load_dataset(dataset_path, ds_config)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

## 3. Exploratory Data Analysis Phase

In [ ]:
import wandb

wandb_apikey = 'd0957427fd4084b53a15acbf9e3abc958c23af93'
wandb.login(key=wandb_apikey)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(project="sentiment-analysis", job_type='upload-dataset')
artifact = wandb.Artifact(name='amazon_reviews_english', type='dataset')


In [ ]:
files = [dataset.cache_files[key][0]['filename'] for key in dataset.cache_files]

for path in files:
    artifact.add_file(local_path=path)


In [ ]:
features = ds_builder.info.features

for key, value in features.items():
    print(key, value)

review_id Value(dtype='string', id=None)
product_id Value(dtype='string', id=None)
reviewer_id Value(dtype='string', id=None)
stars Value(dtype='int32', id=None)
review_body Value(dtype='string', id=None)
review_title Value(dtype='string', id=None)
language Value(dtype='string', id=None)
product_category Value(dtype='string', id=None)


In [ ]:
import pandas as pd

df = pd.DataFrame()
for key in dataset:
    df_tmp = dataset[key].to_pandas()
    df_tmp['split'] = key
    df = pd.concat([df, df_tmp])

df.shape

(210000, 9)

In [ ]:
df_sample = df[df['split'] == 'train'].sample(1000)

table = wandb.Table(columns=list(df_sample.columns), data=df_sample)
artifact.add(table, "eda_table_sample")

ArtifactManifestEntry(path='eda_table_sample.table.json', digest='vEFJF7Q3GzpYyadFHJI1eQ==', ref=None, birth_artifact_id=None, size=300703, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmppne6jotk')

In [ ]:
table = wandb.Table(columns=list(df.columns), data=df)
wandb.Table.MAX_ROWS = df.shape[0] + 1000
artifact.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='0jZvFxk60FhjfQMBbjZXQQ==', ref=None, birth_artifact_id=None, size=60790671, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmpls1a1bn4')

In [ ]:
run.log_artifact(artifact)
run.finish()

## 4. Setup Phase

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

import numpy as np
import pandas as pd

In [ ]:
dataset_path = "amazon_reviews_multi"
ds_config = 'en'

text_col = 'review_body'
label_col = 'stars'


In [ ]:

raw_datasets = load_dataset(dataset_path, ds_config)
raw_datasets = raw_datasets.rename_columns({text_col: 'text', label_col: 'labels'})
drop_columns = list(set(raw_datasets["train"].column_names) - set(['text', 'labels']))
raw_datasets = raw_datasets.remove_columns(drop_columns)

# find number of classes and map 1-5 stars to a range of 0 to 4
number_of_classes = len(pd.unique(raw_datasets['train']['labels']))
class_map = dict(zip(pd.unique(raw_datasets['train']['labels']), 
                     pd.unique(raw_datasets['train']['labels'])-1))

raw_datasets = raw_datasets.map(lambda example: {'labels': class_map[example['labels']]})
raw_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 5000
    })
})

In [ ]:

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenization(example):
    return tokenizer(example['text'], padding=True, truncation=True, 
                     max_length=128, )

tokenized_datasets = raw_datasets.map(tokenization, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets["train"].column_names

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

### PyTorch and Accelerate

In [ ]:
# !pip install accelerate
from torch.utils.data import DataLoader
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
from tqdm.auto import tqdm

In [ ]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
tokenized_datasets.set_format("torch")

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

In [ ]:
def training_function():
    accelerator = Accelerator()

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=number_of_classes)
    optimizer = AdamW(model.parameters(), lr=3e-5)

    train_dl, eval_dl, model, optimizer = accelerator.prepare(
        train_dataloader, eval_dataloader, model, optimizer
    )

    num_epochs = 1
    num_training_steps = num_epochs * len(train_dl)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    progress_bar = tqdm(range(num_training_steps))

    model.train()
    for epoch in range(num_epochs):
        for batch in train_dl:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
    
    return model

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)

### TensorFlow / Keras

In [ ]:
# create tensorflow dataset
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

### Use Src

In [ ]:
from src.config import config
import wandb

wandb.login(key=config.WANDB_KEY)
wandb.login(key=wandb_apikey)

train_config = {
    "batch_size": config.batch_size,
    "epochs": config.epochs,
    "lr": config.learning_rate,
    "pretrained": True,

}

run = wandb.init(project=config.WANDB_PROJECT, job_type="training", config=train_config)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from src.my_datasets import Dataset


ds = Dataset(dataset_path=config.dataset_path, dataset_config=config.dataset_config, 
        config.text_column, config.label_column,
        num_labels=5, tokenizer_checkpoint=config.checkpoint)

train_dataset, validation_dataset, test_dataset = ds.get_dataset(config.batch_size)

model = TFAutoModelForSequenceClassification.from_pretrained(config.checkpoint, 
                                                             num_labels=ds.number_of_classes)

model.compile(
    optimizer=Adam(config.learning_rate),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train the model for one epoch
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=config.epochs,
    callbacks=[SaveModelCallback(monitor='accuracy'),
               WandbCallback(log_preds=False, log_model=True)]
)

In [ ]:
preds = model.evaluate(tf_test_dataset)["logits"]


In [ ]:
wandb.finish()

## 5. Modeling Phase

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

num_train_steps = len(train_dataset) * config.epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=config.learning_rate, end_learning_rate=0.0, 
    decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam


model = TFAutoModelForSequenceClassification.from_pretrained(config.checkpoint, 
                                                             num_labels=ds.number_of_classes)
model.compile(optimizer=Adam(learning_rate=lr_scheduler), 
              loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

In [ ]:
preds = model.predict(validation_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["validation"]["label"])

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200000
})

## 6. Evaluation Phase

In [ ]:
preds = model.predict(tf_test_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["validation"]["label"])

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
from sklearn.metrics import classification_report

classification_report(y_true, y_pred)

## 7. Deployment Phase